# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [1]:
import xlwings as xw
import numpy as np
xw.__version__

'0.24.9'

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [2]:
wb = xw.Book('./07_excel_data/sebestoimostA_v1.xlsx')

In [11]:
sheet = wb.sheets['Рецептура']
consumption = sheet.range('G7:O10').options(np.array).value
consumption
unit_price = sheet.range('G14:O14').options(np.array).value
unit_price
cost = np.nan_to_num(consumption * unit_price).sum(axis=1)
cost

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [12]:
sheet.range('T7:T10').options(transpose=True).value = cost
sheet.range('T6').value = 'Себестоимость'

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [13]:
sheet.range('T4:T6').api.Merge()

4. Выполнить 3 с помощью "протягиваемых" формул.

In [14]:
from xlwings.constants import AutoFillType

In [15]:
sheet.range('V7').formula = '=SUMPRODUCT(G7:O7, $G$14:$O$14)'
sheet.range('V7').api.AutoFill(sheet.range('V7:V10').api, AutoFillType.xlFillDefault)

True

## Лабораторная работа 7

In [1]:
import pandas as pd
import numpy as np
import xlwings as xw
from xlwings.constants import AutoFillType

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
reviews_sample = pd.read_csv('./07_excel_data/reviews_sample.csv', sep=',')
reviews_sample = reviews_sample.rename(columns={"Unnamed: 0": "index"})
reviews_sample = reviews_sample.set_index("index")
# reviews_sample
recipes_sample_with_tags_ingredients = pd.read_csv('./07_excel_data/recipes_sample_with_tags_ingredients.csv', sep=',')
recipes_sample_with_tags_ingredients = recipes_sample_with_tags_ingredients.rename(columns={"Unnamed: 0": "index"})
recipes_sample_with_tags_ingredients = recipes_sample_with_tags_ingredients.set_index("index")
recipes_sample_with_tags_ingredients.drop(["contributor_id", "n_steps","n_tags","tags","ingredients"], axis=1, inplace=True)
# recipes_sample_with_tags_ingredients

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [3]:
new_file = pd.ExcelWriter('recipes.xlsx')
recipes = recipes_sample_with_tags_ingredients.sample(frac = 0.05)
recipes.to_excel(new_file, sheet_name='Рецепты')

reviews = reviews_sample.sample(frac = 0.05)
reviews.to_excel(new_file, sheet_name='Отзывы') 
new_file.save()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [4]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
sheet.range("H1").value = 'seconds_assign'

sheet.range("H2:H1501").options(transpose = True).value = (recipes['minutes'] * 60).to_numpy() # options(transpose=True) для четния и записи

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [5]:
sheet.range("I1").value = 'seconds_formula'
sheet.range("I2").formula = '=D2*60'

sheet.range('I2').api.AutoFill(sheet.range('I2:I1501').api, AutoFillType.xlFillDefault)

True

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [6]:
sheet.range("J1").value = "n_reviews"

n = str(len(reviews) + 1)
sheet.range("J2").formula = f'=COUNTIF(Отзывы!$C$2:$C${n}, "="&B2)'
sheet.range('J2').api.AutoFill(sheet.range("J2:J1501").api, AutoFillType.xlFillDefault)

True

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [7]:
sheet['H1:J1'].api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [11]:
values = sheet.range("D2:D1501").value

for j in range(len(values)):
    if (values[j] < 5):
        sheet.range(f"D{j+2}").color = (0, 255, 0) # зелёный
    elif (5 <= values[j] <= 10):
        sheet.range(f"D{j+2}").color = (255, 255, 0) # жёлтый
    else:
        sheet.range(f"D{j+2}").color = (255, 0, 0) # красный

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
from typing import Union
def validate(data : Union[xw.main.Sheet, xw.main.Book]) -> None:
    
    sheet = None
    if isinstance(data, xw.main.Book):
        try:
            sheet = data.sheets["Отзывы"]
        except Exception as e:
            raise ValueError("Нет отзывов в данном файле Excel!")
    
    elif isinstance(data, xw.main.Sheet):
        sheet = data
    
    else:
        raise ValueError("data должен принимать либо Book, либо Sheet!")

    rng = sheet.range('A1')
    n = rng.current_region.end('down').row

    sheet.range("H2").formula = '=COUNTIF(Рецепты!$B$2:Рецепты!$B$1501,"="&C2)>0'
    sheet.range("H2").api.autofill(type=AutoFillType.xlFillDefault, destination=sheet.range(f"H2:H{n}").api)

    for i in range(2, int(n)+1):
        
        rating = sheet.range(f"E{i}").value
        exist_flag = sheet.range(f"H{i}").value
        
        if rating not in range(0,6) or not exist_flag:
            sheet.range(f"A{i}:G{i}").color = (255, 0, 0)
    
    sheet.range('H:H').value = " "